In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('cleaned_data_final.csv')

In [2]:
df.drop_duplicates('Object ID',keep='first',inplace=True)
df = df.set_index('Object ID')
df = df[df['image_url'].notnull() & df['Title'].notnull()]

Dictionary: Valid object IDs for each category

In [3]:
category_list = ['Artist Display Name','century','Country']
valid_id_dict = dict()
for x in category_list:
    valid_id_dict[x] = df[x].dropna().index.tolist()

Dictionary: unique values for each category

In [4]:
unique_values_dict=dict()
for x in category_list:
    unique_values_list=list(set(df[x].dropna().tolist()))
    unique_values_dict[x]=[]
    for y in unique_values_list:
        if len(str(y))<=50:
            unique_values_dict[x].append(str(y))

Dictionary: question stem for each category

In [5]:
question_stem_dict={'Artist Display Name':'Who created this object?','century':'When was this object created?','Country':'Where was this object created?'}

title, img, link, ques, correct, wrong1,wrong2,wrong3

In [6]:
def gen_question():
    import random
    category=random.choice(category_list)
    valid_id_list=valid_id_dict[category]
    obj_id=random.choice(valid_id_list)
    CORRECT=df.loc[obj_id][category]
    TITLE=df.loc[obj_id]['Title']
    while len(str(CORRECT))>50 or len(str(TITLE))>50:
        obj_id=random.choice(valid_id_list)
        CORRECT=df.loc[obj_id][category]
        TITLE=df.loc[obj_id]['Title']
    QUES=question_stem_dict[category]
    IMG=df.loc[obj_id]['image_url']
    LINK=df.loc[obj_id]['Link Resource']
    if not category=='century':
        CORRECT=str(CORRECT)
        candidates=unique_values_dict[category]
        matching=[s for s in candidates if CORRECT in s]
        other_candidates=[x for x in candidates if not x in matching and len(str(x))<=50]
        [wrong1,wrong2,wrong3]=random.sample(other_candidates,3)
        CHOICES=[CORRECT,wrong1,wrong2,wrong3]
    else:
        CORRECT=int(CORRECT)
        interval=2 
        choices_u_bound=min(CORRECT+interval*random.randint(0,3),21)
        choice_raw_list=[choices_u_bound,choices_u_bound-2,choices_u_bound-4,choices_u_bound-6]
        CHOICES=[]
        for x in choice_raw_list:
            if x >0:
                CHOICES.append(f'A.D. {x}th century')
            else:
                CHOICES.append(f'{x}the century B.C.')
    random.shuffle(CHOICES)
    result=dict()
    result['title']=TITLE
    result['ques']=QUES
    result['img']=IMG
    result['link']=LINK
    result['correct']=CORRECT
    result['choices']=CHOICES
    return result

In [7]:
gen_question()

{'title': 'The Hamlet Of Optevoz',
 'ques': 'Who created this object?',
 'img': 'https://images.metmuseum.org/CRDImages/dp/original/DP823463.jpg',
 'link': 'http://www.metmuseum.org/art/collection/search/333778',
 'correct': 'Charles-François Daubigny',
 'choices': ['Jan Papinck',
  'Charles-François Daubigny',
  'Kekrops Painter',
  'John Bushman']}

In [8]:
from tkinter import *
from tkinter import scrolledtext
import urllib.request
from PIL import Image, ImageTk
import io
from io import BytesIO

#functions to be implemented: connect each page to the randomly generated question
#user can choose from a multiple choice question
#user is told right or wrong
#count the correct answer number
#quit button
class quizGUI:
    
    def __init__(self):
        root = Tk()
        root.lift()
        root.attributes("-topmost", True)
        root.geometry("800x600")
        root.title("MET-Q")
        self.root = root
        color = 'white'
        color1 = 'gray77'
        color2 = 'LightCyan2'
        txtf = 'Verdana 12'
        
        #canvas
        canvas_width = 800
        canvas_height = 600
        canvas = Canvas(root, height=canvas_height, width=canvas_width)
        self.canvas=canvas
        
        #fountain of information
        self.fountain=gen_question()

        #title frame and label
        self.title_frame = Frame(root,height=50)
        self.title_frame.pack(side='top',fill=X)
        self.title_frame.config(bg='red')
        self.title_label = Label(self.title_frame, text="The MET Quiz", font = "Times 30 bold")
        self.title_label.config(bg='red', fg='white')
        self.title_label.pack()
#         self.title_label.pack(side='top',pady=5)
        
        #record the number of questions completed so far
        self.num_var=StringVar()
        self.num=IntVar()
        self.num_var.set('Questions completed: 0')
        self.num_of_ques=Label(root,textvariable=self.num_var,font='Verdana 14')
        self.num_of_ques.config(bg=color)
        self.num_of_ques.pack(side='top',pady=5)
    
        #record the number of correct answers so far
        self.total = IntVar() 
        self.total_var=StringVar()
        self.total_var.set('Number of correct answers: 0')
        self.score_label = Label(root,textvariable=self.total_var,font='Verdana 14')
        self.score_label.config(bg=color)
        self.score_label.pack(side='top',pady=(0,5))
        #to ensure that the user cannot click on other choices after choosing the answer; or click 'more info' before choosing.
        self.already_chosen=0
        
        #correct/incorrect indicator
        var=StringVar()
        self.cor_label=Label(root,textvariable=var,font=txtf,height=2,width=8)
        self.cor_label.config(bg=color)
        self.var=var
        self.cor_label.pack(side='top',pady=(0,5))
    
        #question stem
        self.question_var=StringVar()
        self.Q = Label(root, textvariable=self.question_var)
        self.question_var.set(self.fountain['ques'])
        self.Q.pack(side='top',pady=(0,5))
        self.Q.config(bg=color)

        
        #object title
        self.obj_title_var = StringVar()
        self.obj_title = Label(root,textvariable=self.obj_title_var)
        self.obj_title_var.set(self.fountain['title'])
        self.obj_title.pack(side='top',pady=(0,5))
        self.obj_title.config(bg=color)

        
#         #Question picture
# #         URL=get_question()[1]
#         #URL = 'https://bakingamoment.com/wp-content/uploads/2018/02/IMG_6207-chocolate-molten-lava-cakes-recipe-square.jpg'
#         URL = 'https://images.metmuseum.org/CRDImages/ad/original/204788.jpg'
#         with urllib.request.urlopen(URL) as url:
#             s = url.read()
#             image_file = Image.open(s)
# #         u = urllib.request.urlopen(URL)
# #         raw_data = u.read()
# #         u.close()

# #         image_file = Image.open(BytesIO(raw_data))
#         image_file=image_file.resize((200,200),Image.ANTIALIAS)
#         photo_image = ImageTk.PhotoImage(image_file)
#         self.pic = Label(root,image=photo_image)
#         self.pic.config(bg=color)
#         self.pic.grid(row=4,columnspan=9)

        #picture link
        self.pic=self.fountain['img']
        self.pic_link=Button(root,text='Picture link',font=txtf,command=lambda: self.open_img_page(self.pic))
        self.pic_link.pack(side='top',pady=(10,20))
        self.pic_link.config(bg=color1,activebackground=color2)
        
        #choice buttons
        choices=self.fountain['choices']
        self.correct_answer=self.fountain['correct']
        self.choice_A=StringVar()
        self.choice_A.set(choices[0])
        self.choice_B=StringVar()
        self.choice_B.set(choices[1])
        self.choice_C=StringVar()
        self.choice_C.set(choices[2])
        self.choice_D=StringVar()
        self.choice_D.set(choices[3])
        self.A = Button(root, textvariable=self.choice_A,font=txtf,command=lambda: self.check_correct(self.choice_A.get(),self.correct_answer))
        self.A.pack(side='top',pady=(0,5))
        self.A.config(bg=color1,activebackground=color2)
        self.B = Button(root, textvariable=self.choice_B,font=txtf,command=lambda: self.check_correct(self.choice_B.get(),self.correct_answer))
        self.B.pack(side='top',pady=(0,5))
        self.B.config(bg=color1,activebackground=color2)
        self.C = Button(root, textvariable=self.choice_C,font=txtf,command=lambda: self.check_correct(self.choice_C.get(),self.correct_answer))
        self.C.pack(side='top',pady=(0,5))
        self.C.config(bg=color1,activebackground=color2)
        self.D = Button(root, textvariable=self.choice_D,font=txtf,command=lambda: self.check_correct(self.choice_D.get(),self.correct_answer))
        self.D.pack(side='top',pady=(0,5))
        self.D.config(bg=color1,activebackground=color2)
        self._next=Button(root,text='Next question',font=txtf,command=lambda:self.next_())
        self._next.config(bg=color1,activebackground=color2)
        self._next.pack(side='top',pady=(20,0))

        #more information link
        self.more_info_link=self.fountain['link']
        self.link=Button(root,text='Know more',font=txtf,command=lambda: self.open_info_page(self.more_info_link))
        self.link.pack(side='left',padx=30,pady=0)
        self.link.config(bg=color1,activebackground=color2)
        
        #message: don't cheat
        self.message_var=StringVar()
        self.message=Label(root,textvariable=self.message_var,font=txtf)
        self.message.config(bg=color)
        self.message.pack(side='left',padx=40,pady=0)
        
        #quit button
        self.close_button=Button(root, text="Quit", font=txtf,command=lambda: root.destroy())
        self.close_button.pack(side='right',padx=30,pady=0)
        self.close_button.config(bg=color1,activebackground=color2)
        
        root.configure(background=color)
        canvas.config(bg=color)
        root.mainloop() 

    #add 1 to score if the correct answer is chosen
    def check_correct(self,this_answer,correct_answer):
        if this_answer==correct_answer:
            self.correct()
        else:
            self.incorrect()
                        
    #set up indicator label if correctly chosen
    def correct(self):
        self.var.set('Correct')
        if self.already_chosen==0:
            self.total.set(self.total.get() + 1) 
            self.total_var.set('Number of correct answers: '+str(self.total.get()))
            self.already_chosen=1
    
    def incorrect(self):
        self.var.set('Incorrect')
        self.already_chosen=1
        
    #update gui for the next question
    def next_(self):
        self.fountain=gen_question()
        self.obj_title_var.set(self.fountain['title'])
        self.question_var.set(self.fountain['ques'])
        self.more_info_link=self.fountain['link']
        self.pic=self.fountain['img']
        choices=self.fountain['choices']
        self.correct_answer=self.fountain['correct']
        self.choice_A.set(choices[0])
        self.choice_B.set(choices[1])
        self.choice_C.set(choices[2])
        self.choice_D.set(choices[3])
        self.already_chosen=0
        self.num.set(self.num.get() + 1) 
        self.num_var.set('Questions completed: '+str(self.num.get()))
        self.var.set('')
        self.message_var.set('')
        
    #more info button cannot be clicked before answering
    def open_info_page(self,link):
        if self.already_chosen==1:
            import webbrowser
            webbrowser.open(link)
        else:
            self.message_var.set("You need to choose an answer first.")

    #direct user to webpage
    def open_img_page(self,link):
        import webbrowser
        webbrowser.open(link)

In [9]:
test=quizGUI()